# Payouts API Overview

The Payouts API lets you see a list of deposits and withdrawals from a seller's banking destination.

A banking destination can be a seller's external bank account, a Square checking or savings account (available only in the United States), or a debit card (for instant transfers).

You can use the Payouts API to automatically reconcile transactions to and from the seller's banking destination to ensure that everything adds up. There are two steps in the reconciliation process: comparing the deposit amount from the bank statement to the payout amount and identifying which transactions go into each payout.

The Payouts API doesn’t include data that's older than January 2021. To access this historical data, use the Seller Dashboard.

## Example use case

Consider a scenario where a seller runs several coffee shops across the country. The coffee shops process thousands of card payments per week. The seller has set up an automated payout schedule, whereby Square sends a payout to the seller's bank account at the end of each day. 

You might use the Payouts API to review the summary of all payouts to make sure it matches the seller's bank account balance.

You can also have the following use cases:

* **Processing fees** - Compute processing fees for a payout by iterating through all payout entries. For example, if a payout has four entries: one charge for $100 with a $2 fee, one loan repayment for –$10, one refund for –$50 with a $1 fee, and a loan repayment refund for –$5, the total processing fee amount for this payout is $1. This is calculated as $2 + $0 – $1 + $0.
* **Online marketplace** - An online marketplace app uses the Payouts API to see a list of deposits and withdrawals from a seller's banking destination. This could involve multiple sellers simultaneously reconciling their transactions.
* **Subscription services** - A subscription-based service uses the Payouts API to reconcile regular payments to content creators.
* **Retail refunds** - A retail business uses the Payouts API to track refunds for customers. This could involve the partial or full refund of purchases made in-store or online.
* **Service marketplace** - A service marketplace, such as a home services platform, uses the Payouts API to track payments to service providers after they complete jobs.

## Payout object

The `Payout` object shows the sum of all transactions paid to a seller's banking destination. It's a parent to a set of child payout entries that reflect the balance change for each transaction. 

Each `Payout` object has an `id` field, which uniquely identifies the `Payout` object for your application. It also has an `end_to_end_id` field that might appear on the seller's bank statements. 

{% aside type="info" %}
You can use the `end_to_end_id` field to automate the process of reconciling each payout with the corresponding line item on the seller's bank statement.
{% /aside %}

The `version` field is initially set to 0. For each successful update, Square increases the version number. You can use the `version` field for [optimistic concurrency](build-basics/common-api-patterns/optimistic-concurrency).

```json copy_code
{
  "payout": {
    "id": "po_070431e4-e351-11eb-a8bb-02420a140009",
    "status": "SENT",
    "location_id": "{LOCATION_ID}",
    "created_at": "2021-07-12T20:37:51Z",
    "updated_at": "2021-07-12T20:37:51Z",
    "amount_money": {
      "amount": 457,
      "currency_code": "EUR"
    },
    "destination": {
      "type": "BANK_ACCOUNT",
      "id": "bact:cgvL1yv43VFjexample"
    },
    "version": 1,
    "type": "BATCH",
    "arrival_date": "2021-07-13",
    "end_to_end_id": "L2100000000"
  }
}
```

The following table describes a few of the key fields:

| Field           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|-----------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `amount_money`  | The amount and currency of the payout that is either sent to or pulled from the seller's banking destination. A positive value for the `amount_money` field represents a credit and increases the balance of the seller's banking destination. A negative value for the `amount_money` field represents a debit and decreases the balance of the seller's banking destination.                                                                                                                                                                                                                                                    |
| `created_at`    | The timestamp of when the payout is sent to the banking destination.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| `arrival_date`  | The timestamp of when the seller can expect the funds to arrive in or be taken out of the banking destination (in ISO 8601 local date format).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| `destination`   | The type and ID, where applicable, of the bank to which the payout is made. The following destination types are supported: * BANK_ACCOUNT: External bank outside of Square.  * CARD: A debit card for the transfer. This is only applicable for manual payouts.  * SQUARE_STORED_BALANCE: Square Checking or Savings account (US).                                                                                                                                                                                                                                                                                                |
| `type`          | The type of payout include the following:  * BATCH: Payouts that include a list of payout entries that can be considered settled. * SIMPLE: Arbitrary amount payouts that do not have any payout entries associated with them. However, a  corresponding payout entry is created in the following batch payout to deduct the amount of the simple payout.                                                                                                                                                                                                                                                                         |
| `end_to_end_id` | A unique ID for the payout that might appear on the seller's bank statement. The following guidelines apply to the `end_to_end`_id field: * Automatic transfer setting - The seller must enable the automatic transfer setting for the `end_to_end_id` field to appear. * Manual transfers - The `end_to_end_id` field isn't supported for manual transfers. * Instant transfers - The `end_to_end_id` field is supported for instant transfers only if the seller has enabled automatic transfers. * International banks - Banks located in non-US regions might not include the `end_to_end_id` field on their bank statements. |

## Payout states

A `Payout` object can have one of three states: `SENT`, `PAID`, or `FAILED`. 

The initial status of a payout is `SENT`. If Square doesn't receive any response from the seller's bank for a time period, Square assumes that the payout is successfully deposited and updates the status to `PAID`. 

If the seller's bank account sends a response saying that the payout failed, Square updates the status of the payout entry to `FAILED`. 

If the response from the seller's bank comes back later than Square expects, a payout that is initially marked as `PAID` can transition to `FAILED` because the deposit or withdrawal ultimately failed.

![payout-api-states](//images.ctfassets.net/1nw4q0oohfju/6DpdkMDHr4gtSbRi3gjj5R/d2403eba6dc881b0863d0e47bee54398/payout-api-states.png)

{% aside type="info" %}
You can use webhooks in an application to constantly listen for a state change in the payouts data and notify you when it meets certain conditions. For example, you might use a webhook to set up an email alert for when a payout state changes from `PAID` to `FAILED`. For more information about using webhooks, see [Square Webhooks Overview](webhooks/overview).
{% /aside %}

## Real-world use cases

This section describes a few practical applications for using the Payouts API.

Square access tokens are required to authenticate and authorize API requests. The samples in this section assume that your access token resides in an environment file in your project folder. If you're executing a sample as a standalone Python script, replace `os.environ['SQUARE_ACCESS_TOKEN']` with your access token. You can [find your Sandbox access token](build-basics/access-tokens#get-a-personal-access-token) in the [${DEVELOPER_DASHBOARD_NAME}](${APP_DASHBOARD}).


### Reconciling transactions
You can use the Payouts API to develop an application to automatically reconcile transactions to and from the Square seller's banking destination. Automating accounting reconciliation streamlines financial processes, reduces manual effort, and minimizes the risk of errors in maintaining accurate financial records.

You need to create your own reconciliation logic based on the payout data received from the API. This reconciliation logic is essential for aligning transactions with payouts and ensuring accurate accounting records.

The following code retrieves the payouts for a specific Square location and prints their IDs and amounts. You can use this data as a starting point for creating your own reconciliation logic based on the specific needs of your accounting system.

In [5]:
# Sample Python code for automated accounting reconciliation
from square.client import Client
import os

# Set the access token from environment variable
SQUARE_ACCESS_TOKEN = 'EAAAENYhnjHmVc51jv8fJqxJYy3OqqcMB6yszlkP_BfChRJfoO7ZW_yygPR5HM_n'

# Set the location ID
location_id = 'LGERKFFHS8TFQ'

# Get Payouts API client
payouts_api = Client(access_token=SQUARE_ACCESS_TOKEN, environment='sandbox').payouts

result = payouts_api.list_payouts(location_id = location_id)

if result.is_success():
    payouts = result.body.get('payouts', [])
    for payout in payouts:
        print("Payout ID:", payout.id, "Amount:", payout.amount_money.amount)
elif result.is_error():
    print("Error retrieving payouts: ", result.errors)

/var/folders/xp/shh8n5d50nvfzsvmhwvbgglm0000gn/T/ipykernel_57519/3540804495.py:12: DeprecationWarning: The 'access_token' params are deprecated. Use 'bearer_auth_credentials' param instead.
  payouts_api = Client(access_token=SQUARE_ACCESS_TOKEN, environment='sandbox').payouts


In this code sample, you must set the following parameter:

* **Location ID** - Set the `location_id` to specify the Square location for which the payouts are being retrieved. You can access location IDs through the Locations API or from the **Locations** tab in the [${DEVELOPER_DASHBOARD_NAME}](${APP_DASHBOARD}).

The code does the following:

- **Payouts retrieval** - The code invokes the `list_payouts` method of the Payouts API to fetch a list of payouts linked to the specified location.
- **Iterating through payouts** - A loop iterates through the retrieved payouts. For each payout, you can implement your custom reconciliation logic.

The code outputs the following for each payout in the list:

- **Payout ID** - The unique identifier for the payout.
- **Amount** - The amount of the payout.

In the payout object, you also have access to the following:

- **Currency code** - The currency code of the payout.
- **Status** - The current status of the payout. For example, `SENT`, `PAID`, or `FAILED`. For information see [Payout states](payouts-api/overview#payout-states).
- **Arrival date** - The expected arrival date of the funds in the bank account.

### Get the end_to_end_id field of a payout

The `end_to_end_id` field is a unique ID for the payout that might appear on the seller's bank statement. You can use this field to uniquely identify a `Payout` object for your application and match it with your bank statement. Banks located in non-US regions might not include the `end_to_end_id` field on their bank statements.

In [6]:
# Sample Python code for getting a payout of a seller
from square.client import Client
import os, json

# Set the access token from environment variable
SQUARE_ACCESS_TOKEN = 'EAAAENYhnjHmVc51jv8fJqxJYy3OqqcMB6yszlkP_BfChRJfoO7ZW_yygPR5HM_n'

# Get payouts api client
payouts_api = Client(access_token=SQUARE_ACCESS_TOKEN, environment='sandbox').payouts

payout_id = 'payout_id'
result = payout_api.get_payout(payout_id)
if result.is_success():
    print(json.dumps(result.body, indent=4))
elif result.is_error():
    print(f"Error retrieving payout with id {payout_id}: ", result.errors)

/var/folders/xp/shh8n5d50nvfzsvmhwvbgglm0000gn/T/ipykernel_57519/4049460196.py:9: DeprecationWarning: The 'access_token' params are deprecated. Use 'bearer_auth_credentials' param instead.
  payouts_api = Client(access_token=SQUARE_ACCESS_TOKEN, environment='sandbox').payouts


NameError: name 'payout_api' is not defined

In the code sample, you must set the following parameter:

- **Payout ID** - Set the `payout_id` to specify the payout for which you want to retrieve the `end_to_end_id` field along with other information.

### Real-time payout status monitoring (webhook handling)

You can use webhooks to listen for state changes in payout data, enabling real-time notifications when a payout's state changes (for example, from `PAID` to `FAILED`), allowing for prompt actions and alerts. For more information, see [Payout states](payouts-api/overview#payout-states).

setup.py

Sample Python code for webhook handling
from flask import Flask, request

app = Flask(__name__)

@app.route('/webhook', methods=['POST'])
def webhook_listener():
    data = request.get_json()
    # Implement logic to handle payout state changes
    print("Webhook Received - Payout State Change:", data['event']['type'])
    return 'Webhook Received', 200

if __name__ == '__main__':
    app.run(port=5000)

The `setup.py` script does the following:

- **Flask application setup** - Creates a Flask application and defines an endpoint (`/webhook`) to handle incoming webhook events.
- **Webhook event processing** - The `webhook_listener` function runs when a POST request is received at the `/webhook` endpoint. It does the following:
  * Extracts JSON data from the request, representing the webhook event.
  * Contains logic to handle payout state changes. This sample prints a message indicating the type of payout event received. 
  * Sends a response with the message 'Webhook Received' and a `200` status code to acknowledge the receipt of the webhook event.

In [7]:
#### list_webhook_events.py

# Sample Python code for getting list of event types supported in webhook subscription
from square.client import Client
import os

# Set the access token from environment variable
SQUARE_ACCESS_TOKEN = 'EAAAENYhnjHmVc51jv8fJqxJYy3OqqcMB6yszlkP_BfChRJfoO7ZW_yygPR5HM_n'

# Set the location ID
location_id = 'LGERKFFHS8TFQ'

# Get Webhook Subscriptions API client
webhook_subscriptions_api = Client(access_token=SQUARE_ACCESS_TOKEN, environment='sandbox').webhook_subscriptions

result = webhook_subscriptions_api.list_webhook_event_types()
if result.is_success():
    print("======== Currently supported event types ========")
    for event_type in result.body.get('event_types'):
        print(event_type)
    print("======== Event type metadatas ========")
    for event_metadata in result.body.get('metadata'):
        print(f"{event_metadata.get('event_type')} | Api version introduced: {event_metadata.get('api_version_introduced')} | Release status: {event_metadata.get('release_status')}")

# Specifically for payout, there are three event types we currently support:  "payout.failed", "payout.paid", "payout.sent", but use list_webhook_event_types() whenever you want to subscribe to new event types.

/var/folders/xp/shh8n5d50nvfzsvmhwvbgglm0000gn/T/ipykernel_57519/417225971.py:14: DeprecationWarning: The 'access_token' params are deprecated. Use 'bearer_auth_credentials' param instead.
  webhook_subscriptions_api = Client(access_token=SQUARE_ACCESS_TOKEN, environment='sandbox').webhook_subscriptions


======== Currently supported event types ========
bank_account.created
bank_account.disabled
bank_account.verified
booking.created
booking.custom_attribute.owned.deleted
booking.custom_attribute.owned.updated
booking.custom_attribute.visible.deleted
booking.custom_attribute.visible.updated
booking.custom_attribute_definition.owned.created
booking.custom_attribute_definition.owned.deleted
booking.custom_attribute_definition.owned.updated
booking.custom_attribute_definition.visible.created
booking.custom_attribute_definition.visible.deleted
booking.custom_attribute_definition.visible.updated
booking.updated
card.automatically_updated
card.created
card.disabled
card.forgotten
card.updated
catalog.version.updated
claim_account.set_password
customer.created
customer.custom_attribute.deleted
customer.custom_attribute.owned.deleted
customer.custom_attribute.owned.updated
customer.custom_attribute.public.deleted
customer.custom_attribute.public.updated
customer.custom_attribute.updated
custome

The `list_webhook_events.py` script retrieves the available event types supported in the payout webhook. For more information, see [Webhook Subscriptions API Overview](webhooks/webhook-subscriptions-api).